<a href="https://colab.research.google.com/github/tinacongithub/MPLIA_Ejercicios/blob/main/MPLIA_Ejercicio_adicional_AprendizajeAutomatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador chistecitos

En este ejercicio adicional pretendo replicar este modelo: https://huggingface.co/Tinaenhugging/clasificador-muchocine-Tinasversion partiendo de un dataset de chistes en español. El modelo resultante podría asignar una calificación numérica de manera automatizada en función de qué tan gracioso es un chiste.

In [1]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

Para este ejercicio voy a usar el dataset mrm8488/CHISTES_spanish_jokes, que contiene chistes en español.

In [11]:
from datasets import load_dataset
raw_dataset = load_dataset("mrm8488/CHISTES_spanish_jokes")

Generating train split:   0%|          | 0/2419 [00:00<?, ? examples/s]

Este es el contenido del dataset:

In [12]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords', 'funny', 'category'],
        num_rows: 2419
    })
})

In [13]:
raw_dataset['train'].to_pandas()

,id,text,keywords,funny,category
0,0,"- ¡Rápido, necesitamos sangre!\n- Yo soy 0 pos...",sangre,1,otros
1,1,- ¿Cuál es el mejor portero del mundial? \n- E...,"futbol,porteros",1,otros
2,2,El otro día unas chicas llamarón a mi puerta y...,"dinero,agua",1,otros
3,3,"- Andresito, ¿qué planeta va después de Marte?...",planetas,1,profesiones
4,4,- ¿Por qué Bob Esponja no va al gimnasio? \n- ...,"esponja,gimnasios",1,otros
...,...,...,...,...,...
2414,2414,Una mujer en la consulta del médico.\n-Hola do...,"doctor,ropa",2,profesiones
2415,2415,Error 0094782: No se detecta ningún teclado pu...,"windows,teclado",2,tematicos
2416,2416,Una chica entra a una iglesia con un buen esco...,"escote,pecho,cura",2,tematicos
2417,2417,Un hombre va a hablar con su jefe:\n-Hola veng...,"boda,jefe",2,familia


Uso el mismo modelo que el clasificador muchocine (Electricidad) para tokenizar el texto.

In [14]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "mrm8488/electricidad-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

In [19]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [20]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/2419 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords', 'funny', 'category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2419
    })
})

In [21]:
tokenized_dataset = tokenized_dataset.rename_column('funny','label')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords', 'label', 'category', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2419
    })
})

A continuación, divido el dataset en un conjunto de entrenamiento y en un conjunto de test.

In [22]:
from datasets import DatasetDict,Dataset
new_tokenized_dataset = tokenized_dataset["train"].shuffle()
len_dataset = len(tokenized_dataset["train"])
train_dataset = tokenized_dataset["train"][0:int(len_dataset*0.8)]
test_dataset = tokenized_dataset["train"][int(len_dataset*0.8):]
new_dataset = DatasetDict({"train":Dataset.from_dict(train_dataset),"test":Dataset.from_dict(test_dataset)})


In [23]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Una vez que el data set está listo para entrenar y evaluar, defino el modelo.

In [24]:
from transformers import TrainingArguments
training_args = TrainingArguments("clasificador-chistecitos",evaluation_strategy="epoch")

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at mrm8488/electricidad-base-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ahora defino la función para evaluar al modelo.

In [26]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

Y el objeto trainer para entrenarlo.

In [27]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=new_dataset["train"],
    eval_dataset=new_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

A continuación entreno el modelo.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
